**patch()** can be used as a <span style="color:blue">**decorator**</span>, a <span style="color:blue">**context manager**</span>, or <span style="color:blue">**stand-alone**</span>.

With **patch()**, by default, values are replaced with **MagicMock** instances.

#### Import for all

In [ ]:
from unittest.mock import patch

#### Basic patch

In [ ]:
x = 42

In [ ]:
# same as patch('__main__.x', new='patched_value')
with patch('__main__.x', 'patched_value'):
    print(x)

In [ ]:
x

In [ ]:
# 'new' defaults to MagicMock
with patch('__main__.x'):
    print(x)

In [ ]:
x

#### Test print

In [ ]:
def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url)

In [ ]:
from unittest.mock import patch
from io import StringIO

with patch('sys.stdout', new=StringIO()) as fake_out:
    expected_url = 'protocol://host.domain\n'
    urlprint('protocol', 'host', 'domain')
    assert fake_out.getvalue() == expected_url

#### Patch object, dict,...

In [ ]:
class ProductionClass:
    def method(self):
        pass
    
    def other_method(self):
        print(10)

with patch.object(ProductionClass, 'method', return_value=None) as mock_method:
    thing = ProductionClass()
#     thing.other_method()
    thing.method(1, 2, 3)
mock_method.assert_called_once_with(1, 2, 3)

In [ ]:
foo = {'key': 'value'}
original = foo.copy()
with patch.dict(foo, {'newkey': 'newvalue'}, clear=True):
    assert foo == {'newkey': 'newvalue'}
assert foo == original

In [ ]:
def func(n):
    print('func input:', n)
    return n

#### 3 ways of using patch

In [ ]:
@patch('__main__.func')
def test1(x, mock_func):
    print('x:', x)
    func(x) # Uses patched example.func
    mock_func.assert_called_with(x)  # assert that mock_func was called with parameter x
    
test1(10)

In [ ]:
with patch('__main__.func') as mock_func:
    func(x) # Uses patched example.func
    mock_func.assert_called_with(x)  # assert that mock_func was called with parameter x

In [ ]:
p = patch('__main__.func')
mock_func = p.start()
func(x)
mock_func.assert_called_with(x)  # assert that mock_func was called with parameter x
p.stop()

#### Nested patches

In [ ]:
class SomeClass:
    def method_1(self):
        pass
    def method_2(self):
        pass
    def method_3(self):
        pass
    
@patch.object(SomeClass, 'method_3')
@patch.object(SomeClass, 'method_2')
@patch.object(SomeClass, 'method_1')
def test(mock_1, mock_2, mock_3):
    assert SomeClass.method_1 is mock_1
    assert SomeClass.method_2 is mock_2
    assert SomeClass.method_3 is mock_3
    SomeClass.method_1('foo')
    SomeClass.method_2('bar')
    SomeClass.method_3('foobar')
    return mock_1, mock_2, mock_3
mock1, mock2, mock3 = test()
mock1.assert_called_once_with('foo')
mock2.assert_called_once_with('bar')
mock3.assert_called_once_with('foobar')